
# Projeto NLP: Adeus manual chato! RAG chatbot para motoristas.

🎯 **Objetivo:**  criar um assistente de carro que explique avisos do painel e recomende ações durante a condução, desenvolvendo um chatbot com reconhecimento de contexto. Esse chatbot será integrado ao manual do carro por meio de um LLM, utilizando LangChain e a técnica de Geração Aumentada por Recuperação (RAG) para fornecer respostas precisas e contextuais. Diga adeus aos manuais chatos!

🔗**Fonte:** Esse projeto foi proposto pela trilha de aprendizados do DataCamp sobre aplicativos LLM com LangChain

📰 **Dados:** Manual adaptado do MG ZS, um SUV compacto, armazenado como um arquivo HTML nomeado como `mg-zs-warning-messages.html`.

🔢 **Passos:**
0. Preparativos iniciais
1. Divisão do documento
2. Armazenamento das representações vetoriais (embeddings)
3. Criação do recuperador
4. Definindo o modelo LLM e o prompt template
5. Definindo a cadeia RAG
6. Executando a cadeia RAG


🛠️ **Detalhes técnicos:** Python, LLM, LangChain, OpenAI, Chroma, RAG.


____

## 0. Preparativos iniciais

Para iniciar o projeto, é preciso obter uma conta de **desenvolvedor com a OpenAI** e **criar uma chave API** como uma variável de ambiente segura e **atualizar as bibliotecas** necessárias. As instruções para esses passos estão descritas abaixo.

*  Crie uma conta de desenvolvedor com a OpenAI:
  1. Vá para a página de [inscrição na API](https://platform.openai.com/).
  2. Crie sua conta (você precisará fornecer seu endereço de e-mail e seu número de telefone).
  3. Vá para a [página de chaves API](https://platform.openai.com/settings/profile?tab=api-keys).
  4. Crie uma nova chave secreta. (As vezes é preciso pagar)
  5. Faça uma cópia dela.

* Atualizar as bibliotecas


In [1]:
import subprocess
import pkg_resources

In [5]:
def install_if_needed(package, version):
    '''Função para garantir que o pacote especificado esteja instalado com a versão necessária.'''
    try:
        # Verifique se o pacote está instalado e qual a sua versão
        pkg = pkg_resources.get_distribution(package)
        if pkg.version != version:
            # Se a versão não corresponder, desinstale a versão existente e instale a versão correta
            subprocess.check_call(["pip", "install", f"{package}=={version}"])
    except pkg_resources.DistributionNotFound:
        # Se o pacote não for encontrado, instale a versão necessária
        subprocess.check_call(["pip", "install", f"{package}=={version}"])
    except pkg_resources.VersionConflict as e:
        print(f"Version conflict: {e}")

# Primeiro, trate o problema da versão do `tenacity`
try:
    subprocess.check_call(["pip", "uninstall", "-y", "tenacity"])
    subprocess.check_call(["pip", "install", "tenacity==8.2.2"])
    subprocess.check_call(["pip", "install", "--force-reinstall", "tenacity==8.2.2"])
except subprocess.CalledProcessError as e:
    print(f"Erro durante a instalação do `tenacity`: {e}")

# Em seguida, instale os outros pacotes
install_if_needed("langchain", "0.2.2")
install_if_needed("langchain-openai", "0.1.8")
install_if_needed("langchain-community", "0.2.3")
install_if_needed("unstructured", "0.14.4")
install_if_needed("chromadb", "0.5.0")
install_if_needed("python-dotenv", "1.0.1")

In [6]:
# Definindo a chave API em uma variável"
import os
from dotenv import load_dotenv

load_dotenv('/content/token.env')

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Importando as bibliotecas necessárias
import langchain
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain_openai import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import UnstructuredHTMLLoader

In [7]:
# Carregando o HTML como um load de documentos LangChain
loader = UnstructuredHTMLLoader(file_path="/content/mg-zs-warning-messages.html")
car_docs = loader.load()




## 1. Divisão o documento

Um processo fundamental na implementação do Retrieval Augmented Generation (RAG) é dividir os documentos em partes para armazenamento em um banco de dados vetorial.

* **Divisor de texto de caracteres**: que divide documentos com base em caracteres e mede o comprimento do bloco pelo número de caracteres.

* **Divisão recursiva por caracteres** : o texto é inicialmente dividido em grandes blocos e depois esses blocos são subdivididos em partes menores de acordo com um critério específico. Por exemplo, o texto é dividido em blocos de 1000 caracteres com uma sobreposição de 200 caracteres, preservando parte do contexto entre os segmentos

In [61]:
# Define variables
chunk_size=1000
chunk_overlap=200

# Split the HTML
splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap
)



# Split the document and print the chunks
docs = splitter.split_documents(car_docs)
print(docs)

[Document(metadata={'source': '/content/mg-zs-warning-messages.html'}, page_content='Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Active Speed Limiter Fault Indicates that the active speed limit system has detected a fault. Contact an MG Authorised Repairer as soon as possible. Engine Coolant Temperature High High engine coolant temperature could result in severe damage. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately. Engine Coolant Temperature Sensor Fault Indicates that the engine coolant temperature sensor has failed. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately.'), Document(metadata={'source': '/content/mg-zs-warning-messages.html'}, page_content='Warning Message Procedure Low Oil Pressure In

In [35]:
car_docs[0].page_content

'Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Active Speed Limiter Fault Indicates that the active speed limit system has detected a fault. Contact an MG Authorised Repairer as soon as possible. Engine Coolant Temperature High High engine coolant temperature could result in severe damage. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately. Engine Coolant Temperature Sensor Fault Indicates that the engine coolant temperature sensor has failed. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately.\n\nWarning Message Procedure Low Oil Pressure Indicates that the oil pressure is too low, which may result in severe engine damage. As soon as safety permits, stop the car, switch off the engine and check the engine 

## 2. Armazenamento das representações vetoriais (embeddings)

Com os dados chunckerizados (dividindos em partes), partimos para opara armazenamento e o ingestão em um banco de dados vetorial do Chroma.


Definindo um prompt tenplate para conectar os documentos recuperados e a entrada do usuário e criando uma cadeia de recuperação para que o LLM acesse esses dados externos.

In [41]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [62]:
# Incorpore os documentos em um banco de dados vetorial Chroma persistente
embedding_function = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectorstore = Chroma.from_documents(
    docs,
    embedding=embedding_function,
    persist_directory=os.getcwd()
)

## 3. Criação do recuperador (retriever)

In [63]:
# Configure o armazenamento vetorial como um recuperador
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

## 4. Inicialize o LLM e o prompt template


In [64]:
# Definindo o modelo llm
llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=openai_api_key)

In [73]:
#Criando um prompt template
prompt_template =PromptTemplate( template = """
Você é um assistente para tarefas de perguntas e respostas.
Utilize os seguintes trechos do retrieved context para responder à pergunta.
Se não souber a resposta, diga apenas que não sabe.
Use no máximo três frases e mantenha a resposta concisa.
{context}

Question: {question}
""",input_variable = ['context', 'question'] )

## 5. Defina a cadeia RAG


In [74]:
# Crie uma cadeia para vincular o recuperador (retriever), o prompt template e o modelo llm
rag_chain = ({"context": retriever, "question": RunnablePassthrough()}
            | prompt_template
            | llm)


## 6. Execute a cadeia RAG

In [75]:
# Converse com o RAG
response_1 = rag_chain.invoke("O que devo fazer se a mensagem 'Engine Coolant Temperature High' aparecer no painel do meu carro?")
response_2 = rag_chain.invoke("Como devo reagir se a luz de advertência 'Low Oil Pressure' acender no painel?")
response_3 = rag_chain.invoke("O que significa a mensagem 'ABS Fault' e o que devo fazer?")


In [76]:
response_1.content

"Se a mensagem 'Engine Coolant Temperature High' aparecer no painel do seu carro, você deve parar o veículo em segurança assim que possível, desligar o motor e contatar imediatamente um reparador autorizado da MG."

In [71]:
response_2.content

"Se a luz de advertência 'Low Oil Pressure' acender no painel, você deve, assim que for seguro, parar o carro, desligar o motor e verificar o nível de óleo do motor. Em seguida, entre em contato com um Reparador Autorizado MG o mais rápido possível."

In [72]:
response_3.content

"A mensagem 'ABS Fault' indica que o sistema de freio anti-bloqueio (ABS) falhou e a função ABS está prestes a ser desativada. Você deve consultar um reparador autorizado da MG imediatamente."